## Setup

In [16]:
!pip  install yfinance streamlit ta

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29496 sha256=a5d9e0a8c92ca50c7cbc105eed76f4a4629d974f1f445437eae7ecbae5fc48c7
  Stored in directory: c:\users\12077\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\a1\d7\29\7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\12077\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Imports

In [6]:
import concurrent.futures
import yfinance as yf
import time
import streamlit as st
import pandas as pd
import ta 

## Code

In [7]:
def fetch_stock_data(ticker):
    """Fetch real-time stock data for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(period="1d")
        if data.empty:
            print(f"No data found for {ticker}")
            return None
        return {
            "Ticker": ticker,
            "Price": round(data["Close"].iloc[-1], 2),  # Latest closing price
            "Volume": int(data["Volume"].iloc[-1])  # Latest trading volume
        }
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        return None

In [8]:
# Function to fetch multiple stocks in parallel
def fetch_multiple_stocks(tickers, max_attempts=3):
    outputs = []
    
    for attempt in range(max_attempts):
        try:
            with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
                future_to_ticker = {executor.submit(fetch_stock_data, ticker): ticker for ticker in tickers}
                
                for future in concurrent.futures.as_completed(future_to_ticker):
                    output = future.result()
                    if output:
                        outputs.append(output)
            return outputs  # Successful attempt
        except Exception as e:
            time.sleep(5)
    
    return []  # Failed all attempts

In [9]:
# Example usage: Fetch stock data for multiple tickers
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
stock_data = fetch_multiple_stocks(tickers, max_attempts=3)

print(stock_data)

[{'Ticker': 'AAPL', 'Price': 244.47, 'Volume': 48769700}, {'Ticker': 'GOOGL', 'Price': 183.77, 'Volume': 29878700}, {'Ticker': 'AMZN', 'Price': 226.65, 'Volume': 42913900}, {'Ticker': 'TSLA', 'Price': 354.11, 'Volume': 51455200}, {'Ticker': 'MSFT', 'Price': 409.64, 'Volume': 21384700}]


In [11]:
df = pd.DataFrame(stock_data)
df.head()

,Ticker,Price,Volume
0,AAPL,244.47,48769700
1,GOOGL,183.77,29878700
2,AMZN,226.65,42913900
3,TSLA,354.11,51455200
4,MSFT,409.64,21384700
